
# Fine-tuning Distilbart-Xsum Summarization

---

In [1]:
!nvidia-smi

Tue Apr 19 20:57:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Setup

---

In [2]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [3]:
! pip install transformers
! pip install datasets
! pip install sentencepiece
! pip install rouge_score
! pip install wandb

     |████████████████████████████████| 4.0 MB 12.0 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
     |████████████████████████████████| 6.6 MB 44.2 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 41.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 12.2 MB/s 
     |████████████████████████████████| 1.1 MB 41.4 MB/s 
     |████████████████████████████████| 136 kB 52.3 MB/s 
     |████████████████████████████████| 212 kB 47.4 MB/s 
     |████████████████████████████████| 127 kB 14.0 MB/s 
     |████████████████████████████████| 271 kB 61.9 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 
     |████████████████████████████████| 144 kB 64.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [4]:
import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [5]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Model and tokenizer

---

Hiperparámetros: 

[HF Bart configuration](https://huggingface.co/transformers/_modules/transformers/configuration_bart.html)

[Fairseq Bart](https://github.com/pytorch/fairseq/tree/master/examples/bart)

In [6]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())

True


In [7]:
#Llamado del modelo
# model_name = "google/pegasus-cnn_dailymail"
model_name = "nsi319/legal-led-base-16384"
#Definición de modelo y tokenizador
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Se fijan los parámetros del modelo
model.config.activation_dropout = 0.0
model.config.max_length=500
model.config.min_length=350

print(model.config)

# tokenización
encoder_max_length = 256 
decoder_max_length = 128

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/618M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

LEDConfig {
  "_name_or_path": "nsi319/legal-led-base-16384",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "LEDForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_window": [
    1024,
    1024,
    1024,
    1024,
    1024,
    1024
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_decoder_position_embeddings": 1024,
  "max_encoder_position_embeddin

## Data

---

### Descarga y Preparación de los Datos

### Cargado de Dataset

In [8]:
train_data_txt = datasets.load_dataset("ccdv/govreport-summarization", split="train")
validation_data_txt = datasets.load_dataset("ccdv/govreport-summarization", split="test")

No config specified, defaulting to: gov_report_summarization_dataset/document


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset gov_report_summarization_dataset downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___gov_report_summarization_dataset/document/1.0.0/32c7f04ec577e9dc99b889eff9d64a321a3a00b7332e8733eb32d39bebbdc34f. Subsequent calls will reuse this data.


No config specified, defaulting to: gov_report_summarization_dataset/document
Reusing dataset gov_report_summarization_dataset (/root/.cache/huggingface/datasets/ccdv___gov_report_summarization_dataset/document/1.0.0/32c7f04ec577e9dc99b889eff9d64a321a3a00b7332e8733eb32d39bebbdc34f)


**Preprocess and tokenize**

In [9]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["report"], batch["summary"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Training

---

### Metrics

In [10]:
# Borrowed from https://github.com/huggingface/transformers/blob/master/examples/seq2seq/run_summarization.py

nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

### Training arguments

In [11]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=8,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,  # demo
    per_device_eval_batch_size=4,
    learning_rate=3e-04,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True, #Para métricas ROUGE
    logging_dir="logs",
    logging_steps=1000,
    save_total_limit=3,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

### Train

Wandb integration

In [12]:
wandb

<module 'wandb' from '/usr/local/lib/python3.7/dist-packages/wandb/__init__.py'>

In [13]:
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="LED-Legal_GovReport_FT",
        config={
            "per_device_train_batch_size": training_args.per_device_train_batch_size,
            "learning_rate": training_args.learning_rate,
            "dataset": "govreport",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run_" + current_time

wandb: Currently logged in as: bigbam (use `wandb login --relogin` to force relogin)


Evaluate before fine-tuning

In [14]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 973
  Batch size = 4
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024


Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a mult

{'eval_gen_len': 428.6968,
 'eval_loss': 5.135702133178711,
 'eval_rouge1': 22.6981,
 'eval_rouge2': 7.9794,
 'eval_rougeL': 15.5655,
 'eval_rougeLsum': 20.4454,
 'eval_runtime': 1071.2802,
 'eval_samples_per_second': 0.908,
 'eval_steps_per_second': 0.228}

Train the model

In [15]:
torch.cuda.is_available()
import tensorflow as tf

In [16]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [17]:
#%%wandb
# uncomment to display Wandb charts

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 17517
  Num Epochs = 8
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 35040
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024


Step,Training Loss
1000,4.057800
2000,3.971700
3000,3.916200
4000,3.855500
5000,3.621200
6000,3.538600
7000,3.548900
8000,3.524200
9000,3.421200
10000,3.190400


Se han truncado las últimas 5000 líneas del flujo de salida.
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Inp

TrainOutput(global_step=35040, training_loss=2.82366719267684, metrics={'train_runtime': 28820.7766, 'train_samples_per_second': 4.862, 'train_steps_per_second': 1.216, 'total_flos': 2.3649744045735936e+16, 'train_loss': 2.82366719267684, 'epoch': 8.0})

Evaluate after fine-tuning

In [18]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 973
  Batch size = 4
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024


Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 256 to 1024 to be a mult

{'epoch': 8.0,
 'eval_gen_len': 351.9054,
 'eval_loss': 3.9639694690704346,
 'eval_rouge1': 27.8368,
 'eval_rouge2': 10.3989,
 'eval_rougeL': 17.5863,
 'eval_rougeLsum': 25.3142,
 'eval_runtime': 826.3487,
 'eval_samples_per_second': 1.177,
 'eval_steps_per_second': 0.295}

In [19]:
if WANDB_INTEGRATION:
    wandb_run.finish()

eval/gen_len,█▁
eval/loss,█▁
eval/rouge1,▁█
eval/rouge2,▁█
eval/rougeL,▁█
eval/rougeLsum,▁█
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████


## Evaluation

---

**Generate summaries from the fine-tuned model and compare them with those generated from the original, pre-trained one.**

In [20]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["report"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

test_samples = validation_data_txt.select(range(16))

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]

loading configuration file https://huggingface.co/nsi319/legal-led-base-16384/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/775a96f789715ac82179380a0854f8db01eb763a1e9fd3c6eb1d3fb7fe2772f2.0910396f3f6be78838f485410e47d48729e0c129e351a56b49300856d917d7a5
Model config LEDConfig {
  "_name_or_path": "nsi319/legal-led-base-16384",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "LEDForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "attention_window": [
    1024,
    1024,
    1024,
    1024,
    1024,
    1024
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "gradient_chec

In [21]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            summaries_before_tuning,
        ),
        headers=["Id", "Summary after", "Summary before"],
    )
)
print("\nTarget summaries:\n")
print(
    tabulate(list(enumerate(test_samples["summary"])), headers=["Id", "Target summary"])
)
print("\nSource documents:\n")
print(tabulate(list(enumerate(test_samples["report"])), headers=["Id", "Document"]))

  Id  Summary after                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [22]:

print("\nTarget summaries:\n")
print(
    tabulate(list(enumerate(test_samples["summary"])), headers=["Id", "Target summary"])
)


Target summaries:

  Id  Target summary
----  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [23]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            summaries_before_tuning,
        ),
        headers=["Id", "Summary after", "Summary before"],
    )
)

  Id  Summary after                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [24]:
print(generate_summary(validation_data_txt.select(range(1)), model)[1])

Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024


['The Food and Drug Administration (FDA) and the U.S. Department of Agriculture (USDA) are responsible for ensuring the safety of virtually all domestic and imported food products. FDA and USDA cooperate with states, tribes, and local food safety and public health agencies to carry out their federal responsibilities. FDA and USDA undertake a risk-based approach, in which inspection rates vary depending on the degree to which the agencies inspect facilities. This report examines (1) the extent to which FDA and USDA have assessed the safety of food products, (2) the extent to which FDA and USDA have designed a risk-based inspection approach: and the extent of actions taken to improve oversight of food safety; and actions taken to improve oversight of food safety; and actions taken to improve oversight of food safety; and actions taken to improve oversight; and oversight; and results; and challenges; and (3) FDA and factors that FDA and factors that FDA and USDA has taken to address risks

In [25]:
len(generate_summary(validation_data_txt.select(range(1)), model)[1][0])

Input ids are automatically padded from 256 to 1024 to be a multiple of `config.attention_window`: 1024


1902

In [26]:
from google.colab import files
torch.save(model.state_dict(), 'LED-Legal_GovReport_Short.pth')

# download checkpoint file
files.download('LED-Legal_GovReport_Short.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>